In [ ]:
!pip install pyshp
!pip install geopandas
!pip install --upgrade nbformat
## Ensure running conda install --channel conda-forge cartopy before
!pip install geoplot


!pip install PyBNG


!pip install convertbng

In [ ]:
import geopandas as gpd
import glob
import pandas as pd
from convertbng.util import convert_bng, convert_lonlat



# Grid references for London: https://getoutside.ordnancesurvey.co.uk/guides/beginners-guide-to-grid-references/
# Read SHx files
files = [
#     "SP_RoadNode.shx",
#     "TL_RoadNode.shx",
#     "SU_RoadNode.shx",
#     "TQ_RoadNode.shx",
     "NC_RoadNode.shx"

]

path = "data_storage/data_storage/oproad_essh_gb/data/"

paths = [path+file for file in files]

geo_df = pd.concat(map(gpd.read_file, paths))



#lga_gdf = gpd.read_file('data_storage/data_storage/oproad_essh_gb/data/SP_RoadNode.shx')    # Load the data using Geopandas

In [ ]:
type(geo_df)

In [ ]:
geo_df.head()

In [ ]:
geo_df.dtypes

In [ ]:
geo_df["formOfNode"].unique()

In [ ]:
geo_df["formOfNode"].value_counts()

In [ ]:
geo_df.count()

### Conversion 

In [ ]:
import plotly.express as px
import geopandas as gpd
import plotly.io as pio
pio.renderers.default = 'browser'

sample = geo_df#.sample(n=1000)

sample["longitude"], sample["latitude"] = convert_lonlat(sample.geometry.x, sample.geometry.y)

In [ ]:
fig = px.scatter_mapbox(sample,
    lat=sample.latitude,
    lon=sample.longitude,
    hover_name="formOfNode",
    zoom=14
)

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.write_html("output/road_features.html")
fig.show()

## Find nearest neighbour and distance from it

In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import BallTree

## Create the two GeoDataFrame to replicate your dataset
appart = gpd.GeoDataFrame({
        'geometry': Point(a, b),
        'x': a,
        'y': b,
    } for a, b in zip(np.random.rand(100000), np.random.rand(100000))
)

pharma = gpd.GeoDataFrame([{
        'geometry': Point(a, b),
        'x': a,
        'y': b,
    } for a, b in zip(np.random.rand(3000), np.random.rand(3000))
])

# Create a BallTree 
tree = BallTree(pharma[['x', 'y']].values, leaf_size=2)

# Query the BallTree on each feature from 'appart' to find the distance
# to the nearest 'pharma' and its id
appart['distance_nearest'], appart['id_nearest'] = tree.query(
    appart[['x', 'y']].values, # The input array for the query
    k=1, # The number of nearest neighbors
)
